<a href="https://colab.research.google.com/github/dinabahar/recipe-recommender/blob/master/2_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recipe Recommender 1
In this notebook I built the 1st iteration of the Recipe Recommender using the pre-processed data, `ready.csv`.

---
## DIRECTORY
1. [Import Libraries](#import)
2. [Read Data](#read)
3. [-](#ingredients)
4. [-](#tags)
5. [-](#minutes)

---
<a id='import'></a>
## 1. Import Libraries
Along with Pandas, Numpy and Matplotlib, we are going to import vectorizers. *CountVectorizer* for the `ingredients` list and *MultiLabelBinarizer* for the `tags` list.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

# Using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<a id='read'></a>
## 2. Read data

In [2]:
#df = pd.read_csv('../data/ready.csv')
#df.head(1)

# Using Google Colab

path = "/content/drive/My Drive/recipe-recommender/ready.csv"
df = pd.read_csv(path)
#df = df.sample(30_000, random_state = 42)
df.head(1)

,name,1-day-or-more,15-minutes-or-less,3-steps-or-less,30-minutes-or-less,4-hours-or-less,5-ingredients-or-less,60-minutes-or-less,african,american,amish-mennonite,appetizers,apples,argentine,asian,asparagus,australian,austrian,bacon,baking,bananas,bar-cookies,barbecue,beans,beef,beef-ribs,beef-sausage,beginner-cook,berries,beverages,birthday,biscotti,bisques-cream-soups,black-beans,blueberries,brazilian,bread-machine,breads,breakfast,british-columbian,...,superbowl,swedish,sweet,sweet-sauces,swiss,tarts,taste-mood,technique,tex-mex,thai,thanksgiving,tilapia,to-go,toddler-friendly,tomatoes,tropical-fruit,tuna,turkey,turkey-breasts,turkish,valentines-day,veal,vegan,vegetables,vegetarian,very-low-carbs,vietnamese,water-bath,wedding,weeknight,welsh,white-rice,whole-chicken,whole-turkey,wild-game,wings,winter,yams-sweet-potatoes,yeast,zucchini
0,crab filled crescent snacks,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
path = "/content/drive/My Drive/recipe-recommender/cleaned_recipes.csv"
recipe_df = pd.read_csv(path)
recipe_df.head()

,name,id,minutes,steps,ingredients
0,crab filled crescent snacks,94947,70,"['heat over to 375 degrees', 'spray large cook...","['crabmeat', 'cream cheese', 'green onions', '..."
1,curried bean salad,429010,20,"['drain & rinse beans', 'stir all ingredients ...","['garbanzo beans', 'black beans', 'onion', 'gi..."
2,delicious steak with onion marinade,277542,25,['heat the oil in a heavy-based pan and cook t...,"['olive oil', 'red onion', 'light brown sugar'..."
3,pork tenderloin with hoisin,78450,15,"['cut pork into 1 / 4-inch slices', 'in a larg...","['pork tenderloin', 'soy sauce', 'hoisin sauce..."
4,mixed baby greens with oranges grapefruit and...,80012,15,['in a salad bowl combine the lettuce with the...,"['mixed baby greens', 'oranges', 'grapefruit',..."


Maximum n_samples = 30,000
---

In [0]:
pivot = df.set_index('name')

In [6]:
%%time

df_sparse = sparse.csr_matrix(pivot.fillna(0))

CPU times: user 140 ms, sys: 1.07 ms, total: 141 ms
Wall time: 145 ms


In [7]:
df_sparse.shape

(30000, 381)

Calculate the Pairwise Distances
---

In [8]:
%%time

recommender = pairwise_distances(df_sparse, metric='cosine')

CPU times: user 18.9 s, sys: 3.98 s, total: 22.9 s
Wall time: 22.9 s


In [9]:
recommender_df = pd.DataFrame(recommender, 
                              columns=pivot.index,
                              index=pivot.index)
recommender_df.head()

name,crab filled crescent snacks,curried bean salad,delicious steak with onion marinade,pork tenderloin with hoisin,mixed baby greens with oranges grapefruit and avocado,buttercream frosting,creamy cheesy scrambled eggs with basil,christmas hard candy,pilgrim s cole slaw,creme de menthe chocolate vanilla drink,lamb patties with harissa,hamburger round the pan,szechuan noodles raw vegan,barbecue coca cola sauce,cream of onion and cheese soup,haricots verts salad,chili lover s chili,s mores cookies,bahama breeze jamaican grilled chicken wings,banana walnut oatmeal,black white and red bean salad,amaretto mimosa,green bean pecan and feta salad,nanay s filipino fried chicken,pinkberry,quick berry trifle,hot mulled cider aprs ski slammer,leah chase s green beans with ham and potatoes,lauchkuchen phyllo leek pie,twice baked ranch potatoes,mini chicken breast sliders,roasted green chilies in cumin garlic cream,turkish coffee mocha,updated waldorf classic,coconut chicken clone of cheesecake factory s bang bang chicken,banana rice pudding,sea scallops with sauce meuniere,easy chicken pilaf casserole,sunshine punch,barefoot contessa s caesar salad with pancetta,...,crab custard casserole,pork chops creole,gravy without meat drippings,hot pink fruit dip,zucchini blossoms with boursin and fresh tomato vinaigrette,crunchy apricot toast,bug salad,mistaken lentil dip,kathy s cake mix cookies,turkey a la veracruz,fudge bars,lime orange or lemon curd,lemon garlic roast pork,swiss chicken,cilantro oil guam,peanut butter and chocolate chunk cookies,super bowl five bean chicken chili,new mexico style pork chops,greek chicken kapama,texas fruit,chris s chicken rice,my most favorite brownies,kate s afternoon wine cooler zwt france,summer squash gratin with tomatoes parmesan and fresh thyme,vidalia onion cornbread,italian eggplant aubergine casserole,spooky spiders,smoked sausage with broccoli and cheese,cabbage stew,strawberry cream dip for fruit,cheddar turkey bake oamc,always gone taco dip,easy valley baked egg rolls rsc,romano s macaroni grill penne rustica by todd wilbur,grilled salmon with garlic mayonnaise,farmhouse fresh orange rhubarb pie,mom s apple cake,persian sugar with walnuts rose water topping filling,meatloaf stuffed red peppers,cheesy english muffin appetizer
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
crab filled crescent snacks,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.880477,1.000000,1.000000,1.000000,1.000000,0.927452,1.000000,1.000000,0.817426,1.000000,1.000000,1.000000,0.817426,1.000000,0.620527,1.000000,1.0,1.000000,1.000000,1.0,0.683772,0.904654,0.908713,1.000000,1.000000,0.841886,0.841886,1.000000,0.800000,0.894591,0.908713,1.000000,...,0.543565,1.000000,1.000000,0.920943,1.000000,1.000000,0.884530,1.000000,1.000000,1.000000,0.641431,1.000000,0.900000,0.918350,1.000000,0.870901,0.894591,0.830969,0.908713,1.000000,0.870901,0.717157,1.000000,0.824588,1.000000,0.894591,0.689913,0.908713,1.000000,1.000000,1.000000,1.000000,0.880477,0.824588,0.880477,0.930993,1.00000,1.000000,1.000000,1.000000
curried bean salad,1.0,0.000000,0.836701,1.000000,0.861987,0.894591,1.000000,0.902410,0.902410,0.946162,0.913934,0.928389,0.543565,0.585356,0.878284,0.817426,1.000000,0.870901,1.000000,0.751548,0.478251,0.894591,0.896720,0.937378,1.0,1.000000,0.925464,1.0,1.000000,1.000000,0.925464,0.918350,0.776393,0.870901,1.000000,0.874755,1.000000,0.913934,0.627322,0.817426,...,0.925464,1.000000,0.817426,1.000000,0.669711,0.866667,0.575736,0.641943,0.870901,0.610751,1.000000,0.850929,1.000000,1.000000,1.000000,1.000000,0.827867,1.000000,0.925464,0.902410,1.000000,0.942265,1.000000,0.785166,1.000000,1.000000,0.949363,0.925464,0.933333,0.935450,0.913934,1.000000,1.000000,1.000000,0.902410,0.943656,0.88453,0.937378,1.000000,1.000000
delicious steak with onion marinade,1.0,0.836701,0.000000,0.817426,0.830969,0.870901,1.000000,0.880477,0.880477,0.934062,0.789181,0.736883,0.664590,0.854905,

"Interactive" Recommender System
---

cache the results
key: name of recipes
values: top 10 recipes


In [94]:
# PorK tEnDeRLoiN wIth HoiSin
x = input("Please name the recipe that you've rated highly! ").lower()
find = recommender_df[x].sort_values()[1:2].reset_index(drop=False).loc[0,'name']

recommendation = recipe_df[recipe_df['name']==find].reset_index(drop=True)

recipe = recommendation.loc[0, 'name']
minutes = recommendation.loc[0, 'minutes']
ingredients = time = recommendation.loc[0, 'ingredients']
steps = recommendation.loc[0, 'steps']

print()
print(f"You should try making a {recipe.upper()}!")
print()
print(f"It takes {minutes} minutes.")
print()
print(f"You need {ingredients[1:-1]}.")
print()
print(f"And this is how {steps[1:-1]}.")

Please name the recipe that you've rated highly! pork tenderloin with hoisin

You should try making a GRILLED PORK CHOPS WITH SAT SAUCE!

It takes 5 minutes.

You need 'unsalted dry roasted peanuts', 'canned unsweetened coconut milk', 'water', 'red chili paste', 'fish sauce', 'shallot', 'cider vinegar', 'sugar', 'pork chops', 'cucumber'.

And this is how 'pure all ingredients except pork in a blender', 'transfer to a small heavy saucepan and bring to a simmer , stirring , over medium-low heat', 'thin sauce if desired with 1 to 2 tbsp water', 'prepare grill for direct-heat cooking over medium-hot charcoal', 'pat pork dry , then sprinkle all over with 1 / 2 tsp salt and 1 / 4 tsp pepper', 'oil grill rack , then grill pork , covered only if using a gas grill , turning once , until just cooked through , about 5 minutes total', 'serve with sauce'.
